# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [8]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [10]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E7%94%9F%E6%B4%BB/"  #生活新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [14]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁


In [15]:
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
SCROLL_PAUSE_TIME = 0

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:

    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "lxml")

    new_height = browser.execute_script("return document.body.scrollHeight")

    #已經到頁面底部
    if new_height == last_height:
        break
    last_height = new_height



### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [16]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#

def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [17]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 還有2年才交屋！婆婆干涉「電視別對著腳」調養子宮　媳無奈：只幫出1%訂金

記者黃可昀／綜合報導
有些長輩相當注重居家風水，甚至晚輩買新屋，還會干涉對方家具應該如何擺設。一名女網友透露，日前和先生買了一間預售屋，預計2年後交屋，但婆婆已迫不及待干涉他們家布置，還多次請師傅解籤，提醒她「電視不要對著腳」，才能好好調養子宮，如此密切的關心令她備感壓力。
原PO在臉書社團《爆怨公社》表示，房子都還沒交屋，婆婆已經在干涉房間電視要怎麼擺，「等要準備裝潢時會不會干涉更多？」從對話紀錄來看，婆婆到廟裡求籤詢問居家風水事宜，師傅告訴她「電視不要對著腳」，否則會影響到子宮。原PO回應，房間空間有限，電視不太可能改位置，結果婆婆神回，「沒有關係，到時候先不要放電視」，堅持電視不可擺在床尾，以免影響到她媳婦的子宮健康。
雖然婆婆是基於好意，但夫妻倆自從買了這間新房子，婆婆就不停到附近廟宇拜拜求籤，展現了強大的控制欲，而且新家就位在婆家附近，她擔心2年後搬進去情況會惡化，另她最不解的是，明明新屋99%的費用都是她和先生一起扛，婆婆只出了大約1%，如今卻不斷干涉新家狀況，令她十分不滿。
網友回應，「所以如果要避孕就電視對著腳就好了？」、「應該是想要你早點生小孩啦！生了就不管會不會影響子宮了啦」、「婆婆問什麼就說沒有要裝、沒有要買、沒有要用、沒有要吃」、「我覺得婆婆似乎是擔心身體健康，這樣我覺得還好」、「就跟婆婆說好好好、是是是，轉頭一樣做自己的啦！」
►最新、最強美白好物
----------------------------------------------------------------------

[1] 打破「躺著賺」迷思！一張圖秒懂被動收入　錢少也能花心力打造

▲被動收入能讓人更省力省時的賺錢。（圖／取自Pixabay）
記者曾筠淇／綜合報導
有「收入」才能支撐生活的所有開銷，而收入又可以簡單歸類為主動收入與被動收入兩種，上班所賺來的錢可歸類為前者，至於不需要靠體力、時間來換取金錢則屬於後者。不過，正是因為被動收入不用一直耗費體力與時間，因此常會有「躺著賺即可」的迷思出現，但其實仍需要靠後續的經營來維持才行。
被動收入